In [1]:
import pandas as pd
import numpy as np
import random
import csv
from pdb import set_trace
#%pdb
mypath = ('E:/OneDrive - University College London/load/analysis/')
#mypath = 'C:/Users/nilli lab/Desktop/load/analysis/'

In [2]:
trials = {}

In [4]:
maxi(1)

NameError: global name 'critical_trials' is not defined

In [3]:
def maxi(trials_starting_index):
    df = pd.DataFrame.from_csv(mypath+'all_images_2.csv')
    df.reset_index(drop=False)

    image_names = df.index.tolist()
    cat1 = df['0'].tolist()
    cat2 = df['1'].tolist()
    cat3 = df['2'].tolist()
    cat4 = df['3'].tolist()
    cat5 = df['4'].tolist()
    ts = df['5'].tolist()
    load = df['6'].tolist()
    exemplar = df['7'].tolist()
    possible_images_dict ={z[0]:list(z[1:]) for z in zip(image_names,cat1,cat2,cat3,cat4,cat5,ts,load,exemplar)}
    lista = possible_images_dict.keys()

    # DEFINING TARGET-PRESENT PROBES
    target_present = []
    intermediate = []
    categories = [' bottle', ' horse', ' pottedplant', ' dog',
                  ' cat', ' person', ' aeroplane', ' car', ' chair', ' sofa', ' bird', ' boat']
    odd = categories[::2]
    even = categories[1::2]
    p_load = ['low','high']

    for category in categories:
        for load in p_load:
            while True:
                trial = str(np.random.choice(lista, 1, replace= False))[2:-2]
                if category in possible_images_dict[trial][7] and load in possible_images_dict[trial] \
                    and trial not in intermediate and trial not in target_present:
                    intermediate.append(trial)
                if len(intermediate) == 1:
                    for each in intermediate:
                        target_present.append(each)
                        possible_images_dict[each].append(category)
                    intermediate = []
                    break

    for category in odd:
        for load in p_load:
            while True:
                trial = str(np.random.choice(lista, 1, replace= False))[2:-2]
                if category in possible_images_dict[trial][7] and load in possible_images_dict[trial] \
                    and trial not in intermediate and trial not in target_present:
                    intermediate.append(trial)
                if len(intermediate) == 1:
                    for each in intermediate:
                        target_present.append(each)
                        possible_images_dict[each].append(category)
                    intermediate = []
                    break
    #print len(target_present)
    for n in target_present:
        possible_images_dict[n].append('Target Present')

    target_present_list = [n for n in lista if 'Target Present' in possible_images_dict[n]]


    # DEFINING TARGET-ABSENT PROBES
    target_absent = []
    intermediate = []
    for category in categories:
        for load in p_load:
            while True:
                trial = str(np.random.choice(lista, 1, replace= False))[2:-2]
                if category not in possible_images_dict[trial][7] and load in possible_images_dict[trial] \
                    and trial not in intermediate and trial not in target_absent and trial not in target_present:
                    intermediate.append(trial)
                if len(intermediate) == 1:
                    for each in intermediate:
                        target_absent.append(each)
                        possible_images_dict[each].append(category)
                    intermediate = []
                    break

    for category in even:
        for load in p_load:
            while True:
                trial = str(np.random.choice(lista, 1, replace= False))[2:-2]
                if category not in possible_images_dict[trial][7] and load in possible_images_dict[trial] \
                    and trial not in intermediate and trial not in target_absent and trial not in target_present:
                    intermediate.append(trial)
                if len(intermediate) == 1:
                    for each in intermediate:
                        target_absent.append(each)
                        possible_images_dict[each].append(category)
                    intermediate = []
                    break

    for each in target_absent:
        possible_images_dict[each].append('Target Absent')

    target_absent_list = [n for n in lista if 'Target Absent' in possible_images_dict[n]]


    probe_trials_1 = target_present_list + target_absent_list

    # DEFINING NO-TARGET IMAGES

    for image in lista:
        if image not in target_present and image not in target_absent:
            possible_images_dict[image].append('NA')
            possible_images_dict[image].append('No Target')



    # DEFINING CRITICAL TRIALS
    #Criteria:
    # - images of set load level
    # - images containing a certain category
    # - image is not already in the list of critical trials for this level of load
    #- image is not aready in the list of critical trials (regardless of level of load)

    def critical_trial(load_level):
        global critical_trials
        temp = []
        container = []
        for element in lista:
            if load_level in possible_images_dict[element]:
                container.append(element)
        for category in categories:
            while len(temp) <1 :
                intermediate = str(np.random.choice(container, 1, replace= False))[2:-2]
                if category in possible_images_dict[intermediate] \
                    and intermediate not in temp and intermediate not in critical_trials \
                    and 'Critical' not in possible_images_dict[intermediate]: # this is just a sanity check
                    temp.append(intermediate)
            for each in temp:
                critical_trials.append(each)
            temp =[]

    cycles = 1
    while True:
        critical_trials = []

        critical_trial('low')
        critical_trial('high')

        critical_trials_1 = critical_trials       
        #######################################################
        #appending to image dict
        #######################################################    
        crit_p = 0
        for n in critical_trials:
            if 'No Target' not in possible_images_dict[n]:
                crit_p = crit_p +1

        critical_low_counter = 0
        critical_high_counter = 0
        critical_low_present_probes = []
        critical_low_absent_probes = []
        critical_high_present_probes = []
        critical_high_absent_probes = []
        crit_p = 0

        for n in critical_trials:
            if 'No Target' not in possible_images_dict[n]:
                crit_p = crit_p +1
            if 'No Target' in possible_images_dict[n] and 'low' in possible_images_dict[n]:
                critical_low_counter = critical_low_counter + 1
            if 'No Target' in possible_images_dict[n] and 'high' in possible_images_dict[n]:
                critical_high_counter = critical_high_counter +1
            if 'Target Present' in possible_images_dict[n] and 'low' in possible_images_dict[n]:
                critical_low_present_probes.append(n)# = critical_low_present_probes + 1
            if 'Target Absent' in possible_images_dict[n] and 'low' in possible_images_dict[n]:
                critical_low_absent_probes.append(n)# = critical_low_absent_probes + 1
            if 'Target Present' in possible_images_dict[n] and 'high' in possible_images_dict[n]:
                critical_high_present_probes.append(n)# = critical_high_present_probes + 1
            if 'Target Absent' in possible_images_dict[n] and 'high' in possible_images_dict[n]:
                critical_high_absent_probes.append(n)# = critical_low_absent_probes + 1
        print crit_p
        print critical_low_counter
        print critical_high_counter
        print critical_low_present_probes
        print critical_low_absent_probes
        print critical_high_present_probes
        print critical_high_absent_probes

        #######################################################
        if crit_p == 0 or len(critical_low_present_probes) == len(critical_high_present_probes) and len(critical_low_absent_probes) == \
        len(critical_high_absent_probes): #check if there is an even number of critical probes
            break
            print 'cycles : ',cycles
        else:
            cycles = cycles + 1
            print len(critical_low_present_probes)
            print len(critical_high_present_probes)
            print len(critical_low_absent_probes)
            print len(critical_high_absent_probes)
            print '#######################'
            target_present = []
            target_absent = []
            critical_trials = []
    #######################################################
    for each in critical_trials:
        possible_images_dict[each].append('Critical')
    for each in possible_images_dict:
        if 'Critical' not in possible_images_dict[each]:
            possible_images_dict[each].append('Normal')
    #######################################################
    print 'Probe trials: ', len(probe_trials_1)
    print 'Target Present Probes: ', len(target_present_list)
    print 'Target Absent probes: ', len(target_absent_list)
    print 'Critical trials:', len(critical_trials)
    print 'Critical, low: ' , critical_low_counter
    print 'Critical, high: ' , critical_high_counter
    print 'Are all critical trials unique? ', len(critical_trials) == len(set(critical_trials))
    #######################################################

    print 'Number of critical probes: ' , crit_p
    print 'critical_low_present_probes: ' , len(critical_low_present_probes)
    print 'critical_high_present_probes: ' , len(critical_high_present_probes)
    print 'critical_low_absent_probes: ' , len(critical_low_absent_probes)
    print 'critical_high_absent_probes: ', len(critical_high_absent_probes)
    print 'cycles : ',cycles

    %pdb
    def critical_probes_distribution(starting):
        global critical_low_present_probes, critical_low_absent_probes
        while True:
            p_left = len(starting)
            p_max = p_left
            distribution = []
            for i in range(4):
                if p_left > 0:
                    x = random.randint(0,p_left)
                    distribution.append(x)
                    p_left = p_left - x
                else:
                    distribution.append(0)
            if sum(distribution) == p_max:
                break
        random.shuffle(distribution)
        return distribution


    from copy import deepcopy
    present_crit_distribution = critical_probes_distribution(critical_low_present_probes)
    absent_crit_distribution = critical_probes_distribution(critical_low_absent_probes)
    present_crit_distribution_original = deepcopy(present_crit_distribution)
    absent_crit_distribution_original = deepcopy(absent_crit_distribution)

    print 'present_crit_distribution', present_crit_distribution
    print 'absent_crit_distribution', absent_crit_distribution

    #Defininfg blocks 
    normal_per_block = []
    probes_per_block = []
    present_per_block = []
    absent_per_block = []
    normal_probes_per_block = []
    critical_per_block = []
    present_critical_per_block = []
    absent_critical_per_block = []

    ###############################
    normal_used = []
    present_used = []
    absent_used = []
    critical_used = []
    critical_probes_used = []
    normal = [n for n in lista if 'Normal' in possible_images_dict[n] and 'No Target' in possible_images_dict[n]]
    ###############################
    block = []
    critical_probes_counter = 0 #useless???
    critical_probes_set = critical_low_present_probes + critical_low_absent_probes + critical_high_present_probes + critical_high_absent_probes
    critical_present_set = [n for n in critical_probes_set if 'Target Present' in possible_images_dict[n]]
    critical_absent_set = [n for n in critical_probes_set if 'Target Absent' in possible_images_dict[n]]
    ###############################
    normal_builder = []
    probes = []
    critical = []
    critical_present = []
    critical_absent = []
    ###############################
    modifier = []

    def block_pair(present_n,absent_n):
    # Pick 4 present probes and 5 absent probes
        global critical_per_block,probes_per_block, present_used, absent_used, critical_used, normal_per_block
        global present_crit_distribution, absent_crit_distribution
        global present_critical_per_block, absent_critical_per_block, critical_probes_used, critical_probes_set
        global present_per_block, absent_per_block

        present_crit = present_crit_distribution.pop(0)
        absent_crit = absent_crit_distribution.pop(0)
        block = []

        for load in p_load:
            #PRINTING STUFF
            print 'present_crit_distribution_original: ',present_crit_distribution_original
            print 'absent_crit_distribution_original: ', absent_crit_distribution_original
            print 'present_crit', present_crit
            print 'absent_crit', absent_crit
            print 'present_crit_distribution: ', present_crit_distribution
            print 'absent_crit_distribution: ', absent_crit_distribution
            print 'critical present set: ', critical_present_set
            print 'critical absent set: ', critical_absent_set
            print'---------------------------------------------------'
            normal_availble = [n for n in normal if n not in normal_used]
            print 'normal available:', len(normal_availble)
        # PICKING CRITICAL PROBES    
        #------------------------------------------------------------------------------
            if present_crit != 0:
                present_critical_per_block = []
                while len(present_critical_per_block) < present_crit:
                    crit_probe = str(np.random.choice(critical_present_set,1,replace = False))[2:-2]
                    if crit_probe not in present_critical_per_block and \
                    crit_probe not in critical_probes_used and load in possible_images_dict[crit_probe] and \
                    'Target Present' in possible_images_dict[crit_probe]:

                        present_critical_per_block.append(crit_probe)
                critical_present.append(present_critical_per_block)

            print 'present_critical_per_block:', present_critical_per_block

            if absent_crit != 0:
                absent_critical_per_block = []
                while len(absent_critical_per_block) < absent_crit:
                    crit_probe = str(np.random.choice(critical_absent_set,1,replace = False))[2:-2]
                    if crit_probe not in absent_critical_per_block and \
                    crit_probe not in critical_probes_used and load in possible_images_dict[crit_probe] and \
                    'Target Absent' in possible_images_dict[crit_probe]:

                        absent_critical_per_block.append(crit_probe)
                critical_absent.append(absent_critical_per_block)

            print 'absent_critical_per_block:', absent_critical_per_block  
            print 'critical_present: ',critical_present
            print 'critical_absent: ',critical_absent

            for n in present_critical_per_block + absent_critical_per_block:
                critical_probes_used.append(n)

            print 'critical_probes_used: ', critical_probes_used
        #------------------------------------------------------------------------------
        # PICKING PRESENT AND ABSENT PROBES
            total = present_n + absent_n
            while len(probes_per_block) < (present_n - present_crit):
                probe = str(np.random.choice(target_present_list,1,replace = False))[2:-2]
                if probe not in probes_per_block and probe not in present_used and load in possible_images_dict[probe] \
                and 'Critical' not in possible_images_dict[probe]:
                    probes_per_block.append(probe)

            print 'probes_per_block :', probes_per_block

            while (present_n - present_crit) <= len(probes_per_block) < total - present_crit - absent_crit:
                probe = str(np.random.choice(target_absent_list,1,replace = False))[2:-2]
                if probe not in probes_per_block and probe not in absent_used and load in possible_images_dict[probe] \
                and 'Critical' not in possible_images_dict[probe]:
                    probes_per_block.append(probe)

            print 'probes_per_block :', probes_per_block

            for n in probes_per_block:
                if 'Target Present' in possible_images_dict[n]:
                    present_used.append(n)
                elif 'Target Absent' in possible_images_dict[n]:
                    absent_used.append(n)

            probes.append(probes_per_block)

            print 'probes: ', probes
            present_per_block = [n for n in probes_per_block if 'Target Present' in possible_images_dict[n]]
            absent_per_block = [n for n in probes_per_block if 'Target Absent' in possible_images_dict[n]]
            print 'Present Used in block: ',len(present_per_block)
            print 'Absent Used in block',len(absent_per_block)
        #------------------------------------------------------------------------------    
        #Counting critical probes    
            critical_probes_counter = len(present_critical_per_block + absent_critical_per_block)
            print 'Critical probes counter: ',critical_probes_counter
            print 'Present Critical: ', len(present_critical_per_block)
            print 'Absent Critical: ', len(absent_critical_per_block)

            modifier.append(critical_probes_counter)
        #------------------------------------------------------------------------------
        #Picking critical trials
            while len(critical_per_block) < (3-critical_probes_counter):
                image = str(np.random.choice(critical_trials,1,replace = False))[2:-2]
                if image not in critical_per_block and image not in critical_used \
                    and 'No Target' in possible_images_dict[image] and load in possible_images_dict[image]:
                    critical_per_block.append(image) 
            print 'List of critical trials without probe: '
            for n in critical_per_block:
                critical_used.append(n)
                print possible_images_dict[n]
            #temp = []

            critical.append(critical_per_block)

        #------------------------------------------------------------------------------
        #Picking normal trials
            while len(normal_per_block) < 51 - (3 - critical_probes_counter):
                image = str(np.random.choice(normal,1,replace = False))[2:-2]
                if image not in normal_per_block and image not in normal_used and load in possible_images_dict[image]:
                    normal_per_block.append(image)
            for n in normal_per_block:
                normal_used.append(n)
            print 'normal in this block: ', len(normal_per_block)
            print 'normal used overall: ',len(normal_used)

            normal_builder.append(normal_per_block)
        #----------------------------------------------------------------------------

            print 'critical_probes_used', critical_probes_used 



            if 'sequence_final' in locals():
                print 'sequence_final exists'
            else:
                print 'sequence final does not exist'
                sequence_final = position_finder()
                print sequence_final
            print'---------------------------------------------------'
            print 'END OF BLOCK PROCESSING'
            print'---------------------------------------------------'
            print 'BUILDING THE BLOCK'
            first_element = [x[0] for x in sequence_final]
            second_element = [x[1] for x in sequence_final]
            second_element_counter = 0
            probes_position = []
            for image in range(60):
                #print [x[1] for x in sequence_final if x[0] ==image][0]
                if image in first_element:

                    print 'image', image, second_element[second_element_counter]

                    if second_element[second_element_counter] == 'Target Present':
                        probes_position.append(image)
                        while True:
                            element = str(np.random.choice(present_per_block,1,replace = False))[2:-2]
                            if element not in block:
                                block.append(element)
                                second_element_counter = second_element_counter +1
                                break       
                    elif second_element[second_element_counter] == 'Target Absent':
                        probes_position.append(image)
                        while True:
                            element = str(np.random.choice(absent_per_block,1,replace = False))[2:-2]
                            if element not in block:
                                block.append(element)
                                second_element_counter = second_element_counter +1
                                break   
                    elif second_element[second_element_counter] == 'Critical':
                        while True:
                            element = str(np.random.choice(critical_per_block,1,replace = False))[2:-2]
                            if element not in block:
                                block.append(element)
                                second_element_counter = second_element_counter +1
                                break     
                    elif second_element[second_element_counter] == 'Critical Present':
                        probes_position.append(image)
                        while True:
                            element = str(np.random.choice(present_critical_per_block,1,replace = False))[2:-2]
                            if element not in block:
                                block.append(element)
                                second_element_counter = second_element_counter +1
                                break   
                    elif second_element[second_element_counter] == 'Critical Absent':
                        probes_position.append(image)
                        while True:
                            element = str(np.random.choice(absent_critical_per_block,1,replace = False))[2:-2]
                            if element not in block:
                                block.append(element)
                                second_element_counter = second_element_counter +1
                                break  
                else:
                    while True:
                        element = str(np.random.choice(normal_per_block,1,replace = False))[2:-2]
                        if element not in block:
                            block.append(element)
                            break

            print 'block:', len(block)   
            print'---------------------------------------------------'
            print 'probes_position: ', probes_position
        #-----------------------------------------------------------------------------
            normal_per_block = []
            probes_per_block = []
            critical_per_block = []
            present_critical_per_block = []
            absent_critical_per_block = []
        #-----------------------------------------------------------------------------
        return block, probes_position

    #test = [(5,print [x[1] for x in test if x[0] ==i][0]20),(6,21)]

    def position_finder():
        global present_critical_per_block, absent_critical_per_block, present_per_block, absent_per_block, critical_per_block
        x = len(present_critical_per_block + absent_critical_per_block)
        print 'x', x
        print 'present_critical_per_block', len(present_critical_per_block)
        print 'absent_critical_per_block', len(absent_critical_per_block)
        while True:    
            while True:
                a = [random.randint(1,10) for r in xrange(12-x)]
                if sum(a) <= 60:
                    break

            new_list_1 = []
            acc = 0
            for i in a:
                acc = acc + i
                new_list_1.append(acc)
            new_list_1 = [n - 1 for n in new_list_1]

            if len(present_critical_per_block) == 0 and len(absent_critical_per_block) == 0:
                print 'ciao', len(critical_per_block)

                sequence = ['Target Present'] * len(present_per_block) + ['Target Absent'] * len(absent_per_block) +\
                ['Critical'] * len(critical_per_block)
                sequence = random.sample(sequence, len(sequence))

            if len(present_critical_per_block) != 0 or len(absent_critical_per_block) != 0:
                print 'ciao2', len(critical_per_block)

                sequence = ['Target Present'] * len(present_per_block)  +\
                ['Target Absent'] * len(absent_per_block) +\
                ['Critical'] * len(critical_per_block) +\
                ['Critical Present'] * len(present_critical_per_block) +\
                ['Critical Absent'] * len(absent_critical_per_block)
                sequence = random.sample(sequence, len(sequence))

            sequence_final = zip(new_list_1, sequence)
            #print sequence_final
            #print len(sequence_final)
            gap_check = [n[0] for n in sequence_final if 'Critical' in n[1]] 
            #print gap_check
            container = []
            for i in gap_check:
                elements = [i+1, i+2]
                for element in elements:
                    if element in new_list_1:
                        container.append(element)
            #print 'container', len(container),container
            if len(container) == 0:
                break
            else:
                sequence_final = []

        print '------------------'
        print new_list_1, len(new_list_1)
        print 'gap_check: ', gap_check
        print len(sequence_final)

        for i in sequence_final:
            print i
        return sequence_final

    def block_unpacker(splitting_index,present,absent):
        block, probes_position = block_pair(present,absent)
        low_block = []
        high_block = []
        for i in range(len(block)):
            if i < (len(block)/2):
                low_block.append(block[i])
            else:
                high_block.append(block[i])
        print probes_position
        print len(low_block), len(high_block)
        return low_block, high_block, probes_position

    low_one, high_two, probes_1_3 = block_unpacker(60,4,5)
    low_two, high_one, probes_2_4 = block_unpacker(60,5,4)
    low_four, high_three, probes_5_7 = block_unpacker(60,4,5)
    low_three, high_four, probes_6_8 = block_unpacker(60,5,4)

    #print len(normal_no_probe)
    #print len(normal_no_probe_used)
    #print len(normal_no_probe) == len(set(normal_no_probe_used))
    #print len(target_present)
    #print len(present_used)
    #print len(target_present) == len(set(present_used))
    #print len(target_absent)
    #print len(absent_used)
    #print len(target_absent) == len(set(absent_used))
    #print len(critical_trials)
    #print len(critical_used)
    #critical_probes_counter = 0
    #probes = target_absent + target_present
    #for probe in probes:
    #    if 'Target Present' in possible_images_dict[probe] and 'Critical' in possible_images_dict[probe]:
    #        critical_probes_counter = critical_probes_counter +1
    #    if 'Target Absent' in possible_images_dict[probe] and 'Critical' in possible_images_dict[probe]:
    #        critical_probes_counter = critical_probes_counter +1
    #    
    #print critical_probes_counter

    while True:
        possible_tones = ['tone1','tone2']
        low_one_tones = [np.random.choice(possible_tones) for r in xrange(3)]
        low_two_tones = [np.random.choice(possible_tones) for r in xrange(3)]
        low_three_tones = [np.random.choice(possible_tones) for r in xrange(3)]
        low_four_tones = [np.random.choice(possible_tones) for r in xrange(3)]
        high_one_tones = []
        high_two_tones = []
        high_three_tones = []
        high_four_tones = []
        for each in low_one_tones:
            if each == 'tone1':
                high_one_tones.append('tone2')
            elif each == 'tone2':
                high_one_tones.append('tone1')
        for each in low_two_tones:
            if each == 'tone1':
                high_two_tones.append('tone2')
            elif each == 'tone2':
                high_two_tones.append('tone1')
        for each in low_three_tones:
            if each == 'tone1':
                high_three_tones.append('tone2')
            elif each == 'tone2':
                high_three_tones.append('tone1')
        for each in low_four_tones:
            if each == 'tone1':
                high_four_tones.append('tone2')
            elif each == 'tone2':
                high_four_tones.append('tone1')
        sound_Hz = low_one_tones + low_two_tones + low_three_tones + low_four_tones + high_one_tones + high_two_tones +\
        high_three_tones + high_four_tones

        tone1_counter = 0
        tone2_counter = 0
        for each in sound_Hz:
            if each == 'tone1':
                tone1_counter = tone1_counter + 1
            elif each == 'tone2':
                tone2_counter = tone2_counter + 1

        low_load_tones = [n for n in sound_Hz[0:12]]
        high_load_tones = [n for n in sound_Hz[12:25]]
        low_load_tone1s = [n for n in low_load_tones if n == 'tone1']
        low_load_tone2s = [n for n in low_load_tones if n == 'tone2']
        high_load_tone1s = [n for n in high_load_tones if n == 'tone1']
        high_load_tone2s = [n for n in high_load_tones if n =='tone2']

        if len(low_load_tone1s) == len(low_load_tone2s) and len(high_load_tone1s) == len(high_load_tone2s):
            print low_one_tones
            print high_one_tones
            print '----------------------'
            print low_two_tones
            print high_two_tones
            print '----------------------'
            print low_three_tones
            print high_three_tones
            print '----------------------'
            print low_four_tones
            print high_four_tones
            print '----------------------'
            print 'total n of tones: ', len(sound_Hz)
            print '----------------------'
            print 'total n of 1500 Hz tones: ', tone1_counter
            print 'total n of 2000 Hz tones: ', tone2_counter
            print '----------------------'
            print len(low_load_tone1s) == len(low_load_tone2s)
            print len(low_load_tone1s)
            print len(low_load_tone2s)
            print len(high_load_tone1s) == len(high_load_tone2s)
            print len(high_load_tone1s)
            print len(high_load_tone2s)
            break
    sound_onset = []
    for i in range(48):
        a = random.uniform(0, 0.999)
        a = int(str(a)[2:5])
        #print a
        sound_onset.append(a)

    #------------------------------------------------------------------------------
    # APPEND TO DICT
    sound_index = 0
    for each in possible_images_dict:
        if 'Critical' in possible_images_dict[each]:
            possible_images_dict[each].append(sound_Hz[sound_index])
            possible_images_dict[each].append(sound_onset[sound_index])
            sound_index = sound_index + 1
        elif 'Normal' in possible_images_dict[each]:
            possible_images_dict[each].append('No tone')
            possible_images_dict[each].append(-999)

    blocks = low_one , high_one , high_two , low_two , high_three , low_three , low_four , high_four

    #trials = {}
    trial_n = trials_starting_index
    for b in range(8):
        this_block_images = blocks[b]
        for im in this_block_images:
            trials[trial_n] = possible_images_dict[im]
            trials[trial_n].append(im)
            trial_n += 1

    print len(trials)

In [ ]:
df = pd.DataFrame.from_dict(possible_images_dict)
df= df.T
df.columns = ['Cat.1','Cat.2','Cat.3','Cat.4','Cat.5','TS','Load','exemplar','question','present_absent','trial_type','tone_Hz','tone_onset']
df.TS = df.TS.astype('float')
df.to_csv(mypath + '../Config/randomization_1.csv')
block_1_half = block
probe_series_1_half = probe_series

In [ ]:
##################
# SECOND ITERATION#
##################

In [ ]:
probe_series = probe_series_1_half + probe_series_2_half
block = block_1_half + block_2_half

In [ ]:
with open('../Config/images.csv', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    
    for i in range(16):
        wr.writerow(block[i])
        #wr.writerow(blocks_sequences_list[i])

with open('../Config/probes.csv', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    
    for i in range(16):
        wr.writerow(probe_series[i])
